In [11]:
using Statistics: mean
using BenchmarkTools
using FFTW

In [49]:
function solve_poisson_3d_ppn(f, Nx, Ny, Nz, Δx, Δy, Δz)
    Lx, Ly, Lz = Nx*Δx, Ny*Δy, Nz*Δz
    function mkwaves(N,L)
        k²_cyc = zeros(N, 1)
        k²_neu = zeros(N, 1)

        for i in 1:N
            k²_cyc[i] = (2sin((i-1)*π/N)   /(L/N))^2
            k²_neu[i] = (2sin((i-1)*π/(2N))/(L/N))^2
        end

        return k²_cyc, k²_neu
    end

    # TODO: Create FFTW plan.
    fh = FFTW.fft(FFTW.r2r(f, FFTW.REDFT10, 3), [1, 2])

    kx²_cyc, kx²_neu = mkwaves(Nx, Lx)
    ky²_cyc, ky²_neu = mkwaves(Nx, Lx)
    kz²_cyc, kz²_neu = mkwaves(Nx, Lx)

    kx² = kx²_cyc
    ky² = ky²_cyc
    kz² = kz²_neu

    ϕh = zeros(Complex{Float64}, Nx, Ny, Nz)

    for k in 1:Nz, j in 1:Ny, i in 1:Nx
        @inbounds ϕh[i, j, k] = -fh[i, j, k] / (kx²[i] + ky²[j] + kz²[k])
    end
    ϕh[1, 1, 1] = 0

    # TODO: Create FFTW plan.
    FFTW.r2r(real.(FFTW.ifft(ϕh, [1, 2])), FFTW.REDFT01, 3) / (2Nz)
end

solve_poisson_3d_ppn (generic function with 2 methods)

In [50]:
@inline incmod1(a, n) = a == n ? one(a) : a + 1
@inline decmod1(a, n) = a == 1 ? n : a - 1

function laplacian3d_ppn(f)
    Nx, Ny, Nz = size(f)
    ∇²f = zeros(Nx, Ny, Nz)
    for k in 2:(Nz-1), j in 1:Ny, i in 1:Nx
       ∇²f[i, j, k] = f[incmod1(i, Nx), j, k] + f[decmod1(i, Nx), j, k] + f[i, incmod1(j, Ny), k] + f[i, decmod1(j, Ny), k] + f[i, j, k+1] + f[i, j, k-1] - 6*f[i, j, k]
    end
    for j in 1:Ny, i in 1:Nx
        ∇²f[i, j,   1] = -(f[i, j,     1] - f[i, j,   2]) + f[incmod1(i, Nx), j,   1] + f[decmod1(i, Nx), j,   1] + f[i, incmod1(j, Ny),   1] + f[i, decmod1(j, Ny),   1] - 4*f[i, j,   1]
        ∇²f[i, j, end] =  (f[i, j, end-1] - f[i, j, end]) + f[incmod1(i, Nx), j, end] + f[decmod1(i, Nx), j, end] + f[i, incmod1(j, Ny), end] + f[i, decmod1(j, Ny), end] - 4*f[i, j, end]
    end
    ∇²f
end

laplacian3d_ppn (generic function with 1 method)

In [51]:
function test_3d_poisson_solver_ppn_div_free(Nx, Ny, Nz, Δx, Δy, Δz)
    f = rand(Nx, Ny, Nz)
    f .= f .- mean(f)
    ϕ = solve_poisson_3d_ppn(f, Nx, Ny, Nz, Δx, Δy, Δz)
    laplacian3d_ppn(ϕ) ≈ f
end

test_3d_poisson_solver_ppn_div_free (generic function with 1 method)

In [53]:
test_3d_poisson_solver_ppn_div_free(100, 100, 100)

true

In [47]:
@benchmark test_3d_poisson_solver_ppn_div_free(100, 100, 100)

BenchmarkTools.Trial: 
  memory estimate:  251.79 MiB
  allocs estimate:  5000269
  --------------
  minimum time:     360.236 ms (21.58% GC)
  median time:      363.250 ms (22.65% GC)
  mean time:        368.724 ms (22.80% GC)
  maximum time:     421.358 ms (29.74% GC)
  --------------
  samples:          14
  evals/sample:     1